In [1]:
import sys
if '..' not in sys.path:
    sys.path.append('..')

In [2]:
import pandas as pd
import numpy as np
import torch

from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision.datasets import mnist
from torchvision.transforms import ToTensor
from torch.optim import SGD
from torch.optim.lr_scheduler import ExponentialLR
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader

from tqdm import tqdm

from source.models import LeNet

/home/d.cherniuk/.conda/mark23/lib/python3.9/site-packages/tqdm-4.64.1-py3.9.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = LeNet()
print (model)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (relu4): ReLU()
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (relu5): ReLU()
)


In [79]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    model = model.cuda()
    print ('USE GPU')
else:
    print ('USE CPU')

USE GPU


In [80]:
batch_size = 256
train_dataset = mnist.MNIST(root='../data/MNIST/train', train=True, transform=ToTensor())
test_dataset = mnist.MNIST(root='../data/MNIST/test', train=False, transform=ToTensor())
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
optimizer = SGD(model.parameters(), lr=1e-1)
scheduler = ExponentialLR(optimizer, gamma=0.99)
loss_fn = CrossEntropyLoss()
all_epoch = 100

for current_epoch in tqdm(range(all_epoch)):
    model.train()
    for idx, (train_x, train_label) in enumerate(train_loader):
        if use_gpu:
            train_x = train_x.cuda()
            train_label = train_label.cuda()
        optimizer.zero_grad()
        predict_y = model(train_x.float())
        loss = loss_fn(predict_y, train_label.long())
        if idx % 100 == 0:
            print('idx: {}, loss: {}, lr: {:.4f}'.format(idx, loss.sum().item(), scheduler.get_last_lr()[0]))
        loss.backward()
        optimizer.step()
#     scheduler.step()
    

    all_correct_num = 0
    all_sample_num = 0
    model.eval()
    for idx, (test_x, test_label) in enumerate(test_loader):
        if use_gpu:
            test_x = test_x.cuda()
            test_label = test_label.cuda()
        predict_y = model(test_x.float()).detach()
        predict_y = torch.argmax(predict_y, axis=-1)
        current_correct_num = predict_y == test_label
        all_correct_num += np.sum(current_correct_num.cpu().numpy(), axis=-1)
        all_sample_num += current_correct_num.shape[0]
    acc = all_correct_num / all_sample_num
    print('epoch: {}, accuracy: {:.4f}'.format(current_epoch, acc))

  0%|          | 0/100 [00:00<?, ?it/s]

idx: 0, loss: 2.3066253662109375, lr: 0.1000
idx: 100, loss: 2.3000175952911377, lr: 0.1000
idx: 200, loss: 2.2729179859161377, lr: 0.1000


  1%|          | 1/100 [00:07<13:02,  7.90s/it]

epoch: 0, accuracy: 0.3023
idx: 0, loss: 2.19880747795105, lr: 0.1000
idx: 100, loss: 1.3139128684997559, lr: 0.1000
idx: 200, loss: 0.39360862970352173, lr: 0.1000


  2%|▏         | 2/100 [00:15<12:49,  7.85s/it]

...

epoch: 66, accuracy: 0.9843
idx: 0, loss: 0.00662318617105484, lr: 0.1000
idx: 100, loss: 0.0034386725164949894, lr: 0.1000
idx: 200, loss: 0.031140977516770363, lr: 0.1000


 68%|██████▊   | 68/100 [08:49<04:06,  7.69s/it]

epoch: 67, accuracy: 0.9769
idx: 0, loss: 0.009986327029764652, lr: 0.1000
idx: 100, loss: 0.0025564092211425304, lr: 0.1000
idx: 200, loss: 0.023488134145736694, lr: 0.1000


 69%|██████▉   | 69/100 [08:57<03:57,  7.66s/it]

epoch: 68, accuracy: 0.9773
idx: 0, loss: 0.005689974408596754, lr: 0.1000


In [72]:
torch.save(model, '../models/lenet_mnist_{:.3f}.pkl'.format(acc))